In [2]:
import random
import shutil
import os

test_apple_dir = "model_data/Test/Apples"
test_orange_dir = "model_data/Test/Oranges"

train_apple_dir = "model_data/Train/Apples"
train_orange_dir = "model_data/Train/Oranges"

RAW_DATASET_PATH = "raw_dataset"
RAW_TRAIN_DATASET_PATH = os.path.join(RAW_DATASET_PATH, "Training")
RAW_TEST_DATASET_PATH = os.path.join(RAW_DATASET_PATH, "Test")
DATASET_PATH = "model_data"
TRAIN_DATASET_PATH = os.path.join(DATASET_PATH, "Train")
TEST_DATASET_PATH = os.path.join(DATASET_PATH, "Test")


In [3]:
# ONLY RUN CELL IF NEEDED TO CLEAR ALL THE MODEL_DATA
print(os.getcwd())

if os.path.exists(test_apple_dir):
    shutil.rmtree(test_apple_dir)

if os.path.exists(test_orange_dir):
    shutil.rmtree(test_orange_dir)

if os.path.exists(train_apple_dir):
    shutil.rmtree(train_apple_dir)

if os.path.exists(train_orange_dir):
    shutil.rmtree(train_orange_dir)

os.mkdir(test_apple_dir)
os.mkdir(test_orange_dir)
os.mkdir(train_apple_dir)
os.mkdir(train_orange_dir)

if (os.path.isfile(os.path.join(TRAIN_DATASET_PATH, "apple_out.csv"))):
    os.remove(os.path.join(TRAIN_DATASET_PATH, "apple_out.csv"))

if (os.path.isfile(os.path.join(TRAIN_DATASET_PATH, "orange_out.csv"))):
    os.remove(os.path.join(TRAIN_DATASET_PATH, "orange_out.csv"))




/home/pbhagava/Desktop/EdgeTestbed/edge_testbed/V1/src/sw/test_images


In [4]:
# For training
# Grab 5 Apple images from each type (7*5 = Total 35)
NUM_IMAGES_PER_APPLE = 6
# Grab 35 Orange Images(Total 35)
NUM_IMAGES_PER_ORANGE = NUM_IMAGES_PER_APPLE * 7
# Put in model_data/Test or /Training

# For testing
# Grab 10 apple and 10 orange images (Total 20)
NUM_TEST_IMAGES = 28 #Make sure multiple of 14


In [5]:
def getImages(directory, destination, name, num_files):
    files = os.listdir(os.path.join(directory, name))
    ret = random.sample(files, num_files)

    for file in ret:
        path = os.path.join(directory, name, file)
        shutil.copy(path, destination)
    # print(len(ret))
    return ret

In [6]:
apple_names = ["Apple Braeburn", "Apple Crimson Snow", "Apple Pink Lady", "Apple Red 1", "Apple Red 2", "Apple Red 3", "Apple Red Delicious"]
orange_names = ["Orange"]

In [7]:
apple_train_images = []
apple_test_images = []
orange_train_images = []
orange_test_images = []

# Get apple train images
for apple_name in apple_names:
    apple_train_images.extend(getImages(RAW_TRAIN_DATASET_PATH, TRAIN_DATASET_PATH + "/Apples", apple_name, NUM_IMAGES_PER_APPLE))

# Get orange train images
orange_train_images = getImages(RAW_TRAIN_DATASET_PATH, TRAIN_DATASET_PATH + "/Oranges", "Orange", NUM_IMAGES_PER_ORANGE)

# Get apple test images
for apple_name in apple_names:
    apple_test_images.extend(getImages(RAW_TEST_DATASET_PATH, TEST_DATASET_PATH + "/Apples", apple_name, NUM_TEST_IMAGES//2//7))

# Get orange test images
orange_test_images = getImages(RAW_TEST_DATASET_PATH, TEST_DATASET_PATH + "/Oranges", "Orange", NUM_TEST_IMAGES//2)

In [8]:
print(len(orange_test_images))
print(len(apple_test_images))
print(len(orange_train_images))
print(len(apple_train_images))

14
14
42
42


In [9]:
# Upscale/Rescale Images to 320x240
def convertTo320x240(dir):
    files = os.listdir(dir)
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        dst = os.path.join(dir, f"resized-{fl}")
        os.system(f"convert {pth} -resize 320x240\! {dst}")
        os.remove(pth)
        print(f"ran convert {pth} -resize 320x240\! {dst}")
    return

convertTo320x240(test_apple_dir)
convertTo320x240(test_orange_dir)
convertTo320x240(train_apple_dir)
convertTo320x240(train_orange_dir)



ran convert model_data/Test/Apples/r_92_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_92_100.jpg
ran convert model_data/Test/Apples/r_90_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_90_100.jpg
ran convert model_data/Test/Apples/95_100.jpg -resize 320x240\! model_data/Test/Apples/resized-95_100.jpg
ran convert model_data/Test/Apples/r_39_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_39_100.jpg
ran convert model_data/Test/Apples/r_268_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_268_100.jpg
ran convert model_data/Test/Apples/262_100.jpg -resize 320x240\! model_data/Test/Apples/resized-262_100.jpg
ran convert model_data/Test/Apples/r_91_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_91_100.jpg
ran convert model_data/Test/Apples/r_94_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_94_100.jpg
ran convert model_data/Test/Apples/r_59_100.jpg -resize 320x240\! model_data/Test/Apples/resized-r_59_100.jpg
ran convert mo

In [10]:
str_ = 'ffmpeg -i converted-230_100.jpg -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e \'1/2 "%u " "\\n"\' -v > test.txt'
print(str_)

# ffmpeg -i converted-230_100.jpg -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e '1/2 "%u " "\n"' -v > test.txt

ffmpeg -i converted-230_100.jpg -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e '1/2 "%u " "\n"' -v > test.txt


In [11]:
# Convert to bitstream



# def convertTo565(dir, fruit, to_csv=False):
#     files = os.listdir(dir)
#     for n, fl in enumerate(files):
#         pth = os.path.join(dir, fl)
#         dst = os.path.join(dir, f"565-{fruit}-{n}.txt")
#         if (to_csv):
#             os.system(f'ffmpeg -i {pth} -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e \'1/2 "%u" ", "\' -v > {dst}')
#             os.remove(pth)
#         else:
#             os.system(f'ffmpeg -i {pth} -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e \'1/2 "%u " "\\n"\' -v > {dst}')
#             os.remove(pth)
#         print(f"ran bitstream conversion on {pth} into {dst}")
#     return

def convertTo565(dir, fruit, to_csv=False):
    files = os.listdir(dir)
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        dst = os.path.join(dir, f"565-{fruit}-{n}.bmp")
        
        os.system(f'convert {pth} -define bmp:subtype=RGB565 {dst}')
        os.remove(pth)

        print(f"ran 565 conversion on {pth} into {dst}")
    return

convertTo565(test_apple_dir, "apple")
convertTo565(test_orange_dir, "orange")
convertTo565(train_apple_dir, "apple")
convertTo565(train_orange_dir, "orange")

ran 565 conversion on model_data/Test/Apples/resized-r_92_100.jpg into model_data/Test/Apples/565-apple-0.bmp
ran 565 conversion on model_data/Test/Apples/resized-r_90_100.jpg into model_data/Test/Apples/565-apple-1.bmp
ran 565 conversion on model_data/Test/Apples/resized-95_100.jpg into model_data/Test/Apples/565-apple-2.bmp
ran 565 conversion on model_data/Test/Apples/resized-r_39_100.jpg into model_data/Test/Apples/565-apple-3.bmp
ran 565 conversion on model_data/Test/Apples/resized-r_268_100.jpg into model_data/Test/Apples/565-apple-4.bmp
ran 565 conversion on model_data/Test/Apples/resized-262_100.jpg into model_data/Test/Apples/565-apple-5.bmp
ran 565 conversion on model_data/Test/Apples/resized-r_91_100.jpg into model_data/Test/Apples/565-apple-6.bmp
ran 565 conversion on model_data/Test/Apples/resized-r_94_100.jpg into model_data/Test/Apples/565-apple-7.bmp
ran 565 conversion on model_data/Test/Apples/resized-r_59_100.jpg into model_data/Test/Apples/565-apple-8.bmp
ran 565 conv

In [12]:
import cv2

def get_SURF_features(dir):
    files = os.listdir(dir)
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        dst = os.path.join(dir, f"SURF_{fl}")

        im = cv2.imread(pth)
        
        surf = cv2.xfeatures2.SURF_create(hessian_threshold=500)
        keypoints, descriptors = surf.detectAndCompute(im, None)

        print(descriptors.shape)
        
    return

get_SURF_features(test_apple_dir)
get_SURF_features(test_orange_dir)
get_SURF_features(train_apple_dir)
get_SURF_features(train_orange_dir)

AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2'

In [1]:
import pandas as pd
import csv


# def convertToCSV(dir):
#     files = os.listdir(dir)
#     for n, fl in enumerate(files):
#         pth = os.path.join(dir, fl)
#         print("Reading ", pth)
#         dst = os.path.join(dir, f"CSV_{fl}")
#         p_csv = pd.read_csv(pth)
#         p_csv.to_csv(dst)

def convertToCSV(dir, fruit):
    files = os.listdir(dir)
    lst = []
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        # print("Reading ", pth)
        p_file = open(pth)
        p_lst = []
        for line in p_file:
            f_line = line[:-2]
            f_line = int(f_line)
            p_lst.append(f_line)
        p_file.close()
        # print(p_lst[:50])
        lst.append(p_lst)
        # print(len(p_lst))
    # print(len(lst))
    dst = os.path.join(TRAIN_DATASET_PATH, f"{fruit}_out.csv")
    with open(dst, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(lst)

convertToCSV(train_apple_dir, "apple")
convertToCSV(train_orange_dir, "orange")

        


NameError: name 'train_apple_dir' is not defined

In [12]:
import pandas as pd

df_apple = pd.read_csv(os.path.join(TRAIN_DATASET_PATH, "apple_out.csv"), header=None)
df_orange = pd.read_csv(os.path.join(TRAIN_DATASET_PATH, "orange_out.csv"), header=None)
print(df_apple.shape)
print(df_orange.shape)

(42, 76800)
(42, 76800)


In [13]:
df_apple.insert(0, "fruit", 0)
df_orange.insert(0, "fruit", 1)

In [14]:
print(df_apple.shape)
print(df_orange.shape)

df_combined = pd.concat([df_apple, df_orange])
print(df_combined.shape)
print(df_combined.iloc[:, 0])


(42, 76801)
(42, 76801)
(84, 76801)
0     0
1     0
2     0
3     0
4     0
     ..
37    1
38    1
39    1
40    1
41    1
Name: fruit, Length: 84, dtype: int64


In [15]:
from sklearn.svm import SVC
from micromlgen import port
from sklearn_porter import Porter
import sys

def trainClassifier(X, y):
    clf = SVC(kernel='linear',gamma=0.001).fit(X, y)
    return clf

clf = trainClassifier(df_combined.iloc[:, 1:], df_combined.iloc[:, 0])
# c_code = port(clf, classmap={
#         0: 'apple',
#         1: 'orange'
#     })

porter = Porter(clf, language='c')


c_code = porter.export()
# print(c_code)

In [16]:
original_stdout = sys.stdout
with open('model.h', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(c_code)
    sys.stdout = original_stdout # Reset the standard output to its original value


In [17]:
# from PIL import Image
# from scipy import misc
# import imageio

# def convertBMPToBinaryListHelper(img):
#     im = Image.open(img, mode='r')
#     pixels = list(im.getdata())
#     lst = []
#     for pixel in pixels:
#         r = pixel[0] & 0b11111111
#         g = pixel[1] & 0b11111111
#         b = pixel[2] & 0b11111111
#         rgb = ((r & 0b11111000) << 8) | ((g & 0b11111100) << 3) | ((b & 0b11100000) >> 3)
#         bin_data = bin(rgb)[2:]
#         if (len(bin_data) != 16):
#             print("Error! not 16 bits of data here!", bin_data)
#             print(r, g, b)
#             bin_data = bin_data.zfill(16)
#             print(f"Converted to: {bin_data}")
#         # print(bin_data)
#         lst.append(bin_data)
#     return lst

# def convertBMPToBinaryList(dir):
#     files = os.listdir(dir)
#     for fl in files:
#         pth = os.path.join(dir, fl)
#         dst = os.path.join(dir, f"binary-{fl[:-4]}.txt")
#         bin_list = convertBMPToBinaryListHelper(pth)
#         dest_file = open(dst, "w")
#         dest_file.write(str(bin_list))
#         print(f"Converted {pth} to {dst}")
#         os.remove(pth)
    

# flattened_image = convertBMPToBinaryList("model_data/Test/Oranges")
# # test_lst = convertBMPToBinaryListHelper("model_data/Test/Oranges/orange8.bmp")